In [1]:
passw = ''
#SETUP package imports
#basic imports
import pandas as pd
import numpy as np
import math

#nlp imports
import nltk
import gensim
from gensim.models import Word2Vec
from string import punctuation
from re import sub

nssg = Word2Vec.load('/users/madke/prozhito_sg_nostem.model')
np.random.seed(12)
#sql import
import sqlalchemy

In [2]:
import os

os.chdir("../mysql")

from mysqlfunctions import load_my_sql, resultToList


os.chdir("../keyword_vecs")

connection = load_my_sql(passw)


C:\Users\madke\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [3]:
nv = pd.read_pickle('/users/madke/documents/071719nv.pkl')
total_df = pd.read_pickle('/users/madke/documents/060519qabegen.pkl')

In [4]:
#keyw = ['очерк']
keyw = ['журнал']
#keyw = ['газет', 'журнал']
#keyw =['роман', 'новелл', 'повест']
#keyw =['стихотворен', 'стих']
#keyw = ['эсс', 'манифест']
#keyw =['дневник', 'записк']

In [5]:
##cleaning and getting dnev entry 
keyw_bools = []

for i in nv['nounst']:
    is_keyw = False
    for n in i:
        if n in keyw:
            is_keyw = True
    if is_keyw == True:
        keyw_bools.append(1)
    else:
        keyw_bools.append(0)

nv_entryids = pd.DataFrame({
    
    'keyw_bools': keyw_bools,
    'entry_id': nv['entry_id']
 
})

nv_entryids = nv_entryids[(nv_entryids['keyw_bools'] == 1)]
nv_entryids = nv_entryids.reset_index(drop = True)

In [6]:
fromsql = []
for i in nv_entryids['entry_id']:
    q = resultToList(connection, str(i))
    fromsql.append(q)

fromsql_ided = list(zip(fromsql, nv_entryids['entry_id']))

keyw_sents = []
ids = []
for i in fromsql_ided:
    sents = nltk.sent_tokenize(i[0][0])
    sents = list(enumerate(sents))
    for n in sents:
        q = n[1].split()
        for z in q:
            for f in keyw:
                if f in z:
                    keyw_sents.append(n[1])
                   # print([n[1], f])
                    ids.append(i[1])



In [7]:
ind_df = pd.DataFrame({
    
    'entry_id': ids,
    'text': keyw_sents
    
})
ind_dfskinny = ind_df.drop_duplicates(subset='text', keep='first', inplace=False)

In [8]:
#removing old Russian + buggy entries from website
not_okay = ['52', '196', '421', '465', '489', '750', '827', '1049']
mask = total_df['diarynum'].isin(not_okay)
filtered_df = total_df[~mask]

filtered_df = filtered_df[['diarynum', 'entry_id', 'entry_year']]
filtered_df = filtered_df[filtered_df['entry_year'] > 1900]

okay_ids = [str(i) for i in list(filtered_df['entry_id'])]
nmask = ind_dfskinny['entry_id'].isin(okay_ids)

fil_cat = ind_dfskinny[nmask]
fil_cat = fil_cat.reset_index()

In [9]:
#
cleaned = []
translator = str.maketrans('','', sub( '\?', '', punctuation))
for i in list(fil_cat['text']):
    n = i.split()
    entry = []
    for z in n:
        z = z.lower()
        s = z.translate(str.maketrans(translator))
        entry.append(s)
    cleaned.append(entry)

data_long2 = []
for n in cleaned:
    holder = []
    for z in n:
        if (z[:len(z) - 3] not in keyw) and (z[:len(z) - 2] not in keyw) and (z not in keyw):
            holder.append(z)
    data_long2.append(holder)
    

vectors = np.empty([100,])
for i in data_long2:
    vec = 0
    empty = False
    for n in i:
        if n in nssg.wv.vocab:
            vec = vec + nssg[n]
    if type(vec) == int:
        vec = [None] * 100
        empty = True
    if empty == False:
        vectors = np.vstack((vectors, vec/np.linalg.norm(vec)))
    else:
        vectors = np.vstack((vectors, vec))
        
vecs = np.delete(vectors, (0), axis = 0)

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [10]:
vecs_df = pd.DataFrame(vecs).reset_index(drop = True)
ind = fil_cat.reset_index(drop = True)
vecs_df2 = pd.concat([ind, vecs_df], axis = 1)
vecs_df2 = vecs_df2.drop(['index'], axis = 1)
#vecs_df2.to_csv('./102420_gazetvecs.csv')

In [11]:
vecs_df2.shape

(4686, 102)

In [17]:
for i, row in ind_df.iterrows():
    print(row['text'])
    print()

Книги, журналы, пыль и никому не нужные размышления молодого индивидуалиста, знающего почти все европейские языки.

Ленин создал третий Интернационал потому, что партии второго голосовали за войну, предали рабочий класс — все это прекрасно изложено в речах Ленина и Сталина и в статьях наших журналов, газет, в книгах и пр.

У нее есть некоторые связи с писателями, журналами, газетами, но вся огромная масса, боящаяся фашизма и войны, остается, в общем, без обслуживания.

Организовать в Париже художественный литературный журнал на французском языке, доступный по цене широким массам.

„СССР на стройке“ стоит в Париже четырнадцать франков, тогда как самый дорогой роскошный буржуазный журнал стоит пять франков.

Собрать со всей страны выставку детского творчества: рисунков, игрушек, техники, детских журналов и прочего.

Что касается доставки журналов, то об этом отвечаю Вам официальным письмом».

Глава семьи — известная личность среди издателей, журналистов, актеров.

Дело в том, что в журна

Начал Горбачев] обсуждение нашего (журнального) положения при всех: «Вот представители “Свободной мысли”... Завтра я подпишу, но не нужно думать, что мы возьмем журнал на содержание».

Разговаривали стоя посреди его кабинета (вместе с нами зашел Медведев, оказавшийся вблизи довольно симпатичным человеком): о политике, Ельцине, о журнале.

Пускаться в длительные объяснения было уже невозможно; пытаясь сохранить какую-то возможность для продолжения обсуждения судьбы журнала, я сказал, что мы напишем свои предложения... Домой я приехал без двадцати десять, все уже беспокоились, куда это я подевался.

Сижу и ломаю голову, как быть: оставаться и разделить судьбу своих товарищей, выпуская журнал, пока совсем не потонем?

Картина с журналом шла 3 часа.

Днем — урок с Володей, разговоры с Пузановыми, полтора часа в читальне (современные журналы).

И снова во мне оживает искра социального альтруизма; статьи о современности в соц[иал]-демокр[атических] журналах («Наша заря» и «Возрождение») шеве


Написал предисловие к документальной публикации «Протоколы ЦК партии октябристов», которую совместно с Д. Б. Павловым подготовили для журнала «Вопросы истории».

У меня все же теплится надежда найти средства на издание протоколов кадетов, их публикация началась в отечественных журналах.

Семинар проходит на базе журнала «Открытая политика» (17.10), его ведет В. Ф. Антонов.

Идея такого семинара хорошая, в журнале оперативно будут публиковаться доклады и сообщения.

Подготовил материал для круглого стола в журнале «Новая и новейшая история» «Западноевропейские и российские политические партии (сравнительный анализ)», статью для журнала «Поиск» «Политические партии России.

Подготовил материал для круглого стола в журнале «Новая и новейшая история» «Западноевропейские и российские политические партии (сравнительный анализ)», статью для журнала «Поиск» «Политические партии России.

Когда моя статья о Велиховых была опубликована в журнале «Вопросы истории», я подарил номер Евгению Павлови

Много читал: газеты, журналы, мог следить за «ходом мысли» теперешней прессы.

И люди действительно пишут, что думают, никого не боясь и ни на кого не оглядываясь... в меру своего писательского и журналистского дара.

Кстати, и талантов журналистских появилась уйма: как грибы после благотворного дождя.

Между прочим, как раз сейчас с первых номеров журналов нового года — обозначился поворот.

Напомнил о предложении Анатолия Иванова на встрече с писателями (надо бы новое Постановление о журналах «Звезда» и «Ленинград»).

Стоило ему бывало назвать фамилию автора или обозревателя, и тот исчезал со страниц журналов и с экрана.

Две недели, как на досуге думаю, а ничего, собственно, не придумал, кроме того, что уже есть в газетах и журналах.

Записывать подробно, как в былые времена, нет сил: за день столько понаписываешь, а приходишь домой в 9–10 часов вечера измочаленный, а тут надо читать газеты и журналы.

Мало разве разного печатают в газетах, в журналах.

Сижу, говорит, обложенный жур

*** Из «Криптограмм [Востока]» можно печатать отдельными легендами в газете или целым собранием в газетах, журналах, по совету Н.К.

Пару статей за год в журналах.

продиктовал письмо-статью для журнала Келлога.

<...>    
Вечером у него был на ужине Зиманд и сказал Н.К., что ему теперь нужно быть осторожным при поездке в Лех, ибо англичане следят [за ним], и вообще, чтобы он ничего русского с собой не брал: книг, журналов.

о том, что никогда нельзя посылать из нашей Школы статьи в журналы или газеты, не имея раньше личное свидание с издателем журнала, вообще личные отношения, иначе будут отсылать не читая, как это и случалось, когда Грант прямо без разбора посылала статьи в журналы.

о том, что никогда нельзя посылать из нашей Школы статьи в журналы или газеты, не имея раньше личное свидание с издателем журнала, вообще личные отношения, иначе будут отсылать не читая, как это и случалось, когда Грант прямо без разбора посылала статьи в журналы.

о том, что никогда нельзя посылать из н

Прочла путешествие Лемпорта по Нигерии в журнале «Искусство кино».

Морис Лейти из Би-би-си написал статью о публикациях в журнале «Коммунист»<com id="14736207114562"/> у нас, в которых некоторые генералы полностью реабилитировали Сталина.

Мы рассматривали фотографии в журналах *Paris* *Match<com id="147362070641"/>* об августовских днях в Праге и Братиславе.

Вчера нам по почте пришла бандероль из Югославии с двумя журналами, посвященными целиком советской литературе<com id="14736207064483"/>.

В журнале четыре вклейки — три Димины скульптуры: «Праздник», «Эйнштейн», «Памятник Варге» и портрет А.И.

В итоге Лиза сказала, что она в Вене работает в журнале *Tagebuch*<com id="14736207024412"/> и хотела бы взять у Димы фотографии, чтобы там что-нибудь опубликовать.

Позвонила Инна Бернштейн, она сейчас ежедневно ходит на работу переводить чехословацкие газеты и журналы для начальства.

Коротенькое письмецо прислала Велта из Риги, просит послать ей список всех книг и журналов с Димиными и

[...]    
>Пользуюсь случаем послать Вам сравнительно недавнюю статью в журнале «Искусство кино»<com id="152879702314464"/>.

Она добилась того, что о выставке есть публикация в каких-то двух журналах и будет каталог.

Положил с нынешнего дня писать в журнале моем непременно каждый день хотя одно слово.

Ровно пять месяцев не писал я в моем журнале.

//«Гражданин»// помещает заметку, сообщающую, что графиня Толстая, супруга графа Льва Николаевича, категорически отрицает передачу какого-либо письма в английские газеты, //«а признает то, что было напечатано в .»Московских ведомостях», искаженным текстом статьи графа Толстого, напечатанной, но не выпущенной в свет в московском журнале «Психология»//.

А еще в этой борьбе неизбежна великая и беспрерывная ложь, которая льется из наших радиоприемников и со страниц многомиллионных тиражей газет, журналов... И праздники!

Мною предложено совсем выключить эти правила из положения, а взамен их сделать оговорку в журнале присутствия.

Государь пр

Я иду на улицу разбираю журналы в ларьках и все ни как не могу остановится на чем нибудь определенном ~~во~~ мне не думается о докладе.

К списку на подписку журналов и газет.

Его привлекает всё необычное, он покупает странные книги, журналы и газеты.

Дебютировал в журнале Московский вестник (1827), публиковал переводы с испанского (Кальдерон де ла Барка, также писал об Испании), английского (повесть Байрона «Вампир»), переводил Шекспира (не опубликовано, рукописи утрачены).

Печатался также в журнале брата «Европеец».

При жизни материалы фольклорного собрания Петра Киреевского публиковались в ЧОИДР, «Московском сборнике», «Русской беседе» и других журналах, но были напечатаны далеко не полностью.

Сборники его стихов, рассыпанных по журналам и газетам, появились лишь посмертно.

О нём пишут в газетах и журналах.

Я почитала, сидя на концерте, разные журналы, где говорится о А.Н.

Н. предлагает мне печататься в интернет журнале.

Он предложил мне напечататься в журнале «Грани».

Я л

Надо мне посмотреть его работы, его публикации, книги, журналы, его архив, публикации о нем основательно, и собирать воспоминания о нем, прежде всего в Нукусе, потом в Ташкент, потом в Москве.

Говорили о реакционном журнале «Молодая гвардия».

Долгую ночь просматривал журналы «В мире книг» скучные, вырывал цветные картинки и нужное по искусству за 74, 75, 76, 77, 78, 79, 80 81, 82 года.

Читал журналы «Свидетелей Иеговы», их можно читать ради ценной информации, имеющейся в них, если осторожно отклонять их ошибки, вроде отрицания бессмертия души, Троицы, Христа.

Отнес Вале «Жития святых» за январь, Детскую Библию и комиксы Бидструпа Наденьке, журналы «Курьер» Мише.

Вспоминаю, как она распродала в макулатуру сотни моих ценнейших книг, а, главное, письма Савицкого мне, научные книги и журналы, целыми ящиками и многие другие мои вещи из квартиры, пока я уходил из дома на работу.

Кроме того, я много книг и журналов сдал в макулатуру.

Ценнейшие письма Савицкого мне, научные книги и журн